In [4]:
import cv2
import numpy

PREVIEW  = 0  # Preview Mode
BLUR     = 1  # Blurring Filter
FEATURES = 2  # Corner Feature Detector
CANNY    = 3  # Canny Edge Detector

feature_params = dict(maxCorners=500, qualityLevel=0.2, minDistance=15, blockSize=9)

# Force webcam 0 (ignore sys.argv because Jupyter breaks it)
source = cv2.VideoCapture(0)

if not source.isOpened():
    print("ERROR: Cannot open webcam")
    raise SystemExit

image_filter = PREVIEW
alive = True

win_name = "Camera Filters"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
result = None

while alive:
    has_frame, frame = source.read()
    if not has_frame:
        break

    frame = cv2.flip(frame, 1)

    if image_filter == PREVIEW:
        result = frame
    elif image_filter == CANNY:
        result = cv2.Canny(frame, 80, 150)
    elif image_filter == BLUR:
        result = cv2.blur(frame, (13, 13))
    elif image_filter == FEATURES:
        result = frame.copy()
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners = cv2.goodFeaturesToTrack(frame_gray, **feature_params)
        if corners is not None:
            for x, y in numpy.float32(corners).reshape(-1, 2):
                cv2.circle(result, (int(x), int(y)), 10, (0, 255, 0), 1)

    cv2.imshow(win_name, result)

    key = cv2.waitKey(1)
    if key in [ord("q"), ord("Q"), 27]:  # ESC or Q
        alive = False
    elif key in [ord("c"), ord("C")]:
        image_filter = CANNY
    elif key in [ord("b"), ord("B")]:
        image_filter = BLUR
    elif key in [ord("f"), ord("F")]:
        image_filter = FEATURES
    elif key in [ord("p"), ord("P")]:
        image_filter = PREVIEW

source.release()
cv2.destroyAllWindows()
